# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels]

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels]

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 122
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            mean_encoder, logvar_encoder = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = mean_encoder + K.exp(logvar_encoder / 2) * epsilon
            return z

        z = Lambda(get_distrib)([mean_encoder, logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None)(hidden_decoder)

def get_loss(x, x_):
    xent_loss = input_dim * metrics.binary_crossentropy(x, x_) 
    kl_loss = - 0.5 * K.sum(1 + logvar_encoder - K.square(mean_encoder) - K.exp(logvar_encoder), axis=-1)
    
    return K.abs(K.mean(xent_loss + kl_loss + label_loss))



In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [4, 16, 32]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [50]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    vae_model = Model(inputs = Train.x, outputs = Train.x_ )
    vae_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ['accuracy'] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  #validation_data = (x_valid[:valid_size,:], x_valid[:valid_size,:]),
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = x_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = x_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)


 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 1s - loss: 0.8667 - acc: 0.0529 - val_loss: 1.7562 - val_acc: 0.1637
Epoch 2/50
112720/112720 [==============================] - 0s - loss: 0.6748 - acc: 0.4359 - val_loss: 1.5903 - val_acc: 0.6757
Epoch 3/50
112720/112720 [==============================] - 0s - loss: 0.5707 - acc: 0.6723 - val_loss: 1.4343 - val_acc: 0.7369
Epoch 4/50
112720/112720 [==============================] - 0s - loss: 0.4954 - acc: 0.7938 - val_loss: 1.3161 - val_acc: 0.7855
Epoch 5/50
112720/112720 [==============================] - 0s - loss: 0.4391 - acc: 0.8329 - val_loss: 1.2154 - val_acc: 0.7904
Epoch 6/50
112720/112720 [==============================] - 0s - loss: 0.3891 - acc: 0.8462 - val_loss: 1.1568 - val_acc: 0.7949
Epoch 7/50
112720/112720 [==============================] - 0s - loss: 0.3568 - acc: 0.8542 - val_loss

112720/112720 [==============================] - 1s - loss: 0.7869 - acc: 0.1091 - val_loss: 1.7773 - val_acc: 0.2033
Epoch 12/50
112720/112720 [==============================] - 1s - loss: 0.7780 - acc: 0.1367 - val_loss: 1.7657 - val_acc: 0.2098
Epoch 13/50
112720/112720 [==============================] - 1s - loss: 0.7697 - acc: 0.1437 - val_loss: 1.7567 - val_acc: 0.2302
Epoch 14/50
112720/112720 [==============================] - 1s - loss: 0.7620 - acc: 0.1497 - val_loss: 1.7490 - val_acc: 0.1834
Epoch 15/50
112720/112720 [==============================] - 1s - loss: 0.7551 - acc: 0.1604 - val_loss: 1.7439 - val_acc: 0.1898
Epoch 16/50
112720/112720 [==============================] - 1s - loss: 0.7486 - acc: 0.1827 - val_loss: 1.7381 - val_acc: 0.2015
Epoch 17/50
112720/112720 [==============================] - 1s - loss: 0.7425 - acc: 0.2018 - val_loss: 1.7352 - val_acc: 0.2075
Epoch 18/50
112720/112720 [==============================] - 1s - loss: 0.7367 - acc: 0.2129 - val_los

112720/112720 [==============================] - 2s - loss: 0.1104 - acc: 0.9328 - val_loss: 0.7518 - val_acc: 0.8737
Epoch 33/50
112720/112720 [==============================] - 2s - loss: 0.1446 - acc: 0.9231 - val_loss: 0.7542 - val_acc: 0.8824
Epoch 34/50
112720/112720 [==============================] - 2s - loss: 0.1176 - acc: 0.9304 - val_loss: 0.7851 - val_acc: 0.8796
Epoch 35/50
112720/112720 [==============================] - 2s - loss: 0.1536 - acc: 0.9235 - val_loss: 0.7604 - val_acc: 0.8456
Epoch 36/50
112720/112720 [==============================] - 2s - loss: 0.1556 - acc: 0.9245 - val_loss: 0.5751 - val_acc: 0.8612
Epoch 37/50
112720/112720 [==============================] - 3s - loss: 0.1726 - acc: 0.9097 - val_loss: 0.7656 - val_acc: 0.8690
Epoch 38/50
112720/112720 [==============================] - 3s - loss: 0.1656 - acc: 0.9212 - val_loss: 0.8156 - val_acc: 0.8864
Epoch 39/50
112720/112720 [==============================] - 2s - loss: 0.1353 - acc: 0.9244 - val_los

112720/112720 [==============================] - 4s - loss: 0.8066 - acc: 0.1131 - val_loss: 1.8052 - val_acc: 0.2453
Epoch 3/50
112720/112720 [==============================] - 4s - loss: 0.7414 - acc: 0.4150 - val_loss: 1.7942 - val_acc: 0.3581
Epoch 4/50
112720/112720 [==============================] - 4s - loss: 0.7175 - acc: 0.4996 - val_loss: 1.7236 - val_acc: 0.6078
Epoch 5/50
112720/112720 [==============================] - 4s - loss: 0.6878 - acc: 0.5399 - val_loss: 1.7575 - val_acc: 0.5983
Epoch 6/50
112720/112720 [==============================] - 4s - loss: 0.6634 - acc: 0.6293 - val_loss: 1.6946 - val_acc: 0.6643
Epoch 7/50
112720/112720 [==============================] - 4s - loss: 0.6390 - acc: 0.6786 - val_loss: 1.8440 - val_acc: 0.3736
Epoch 8/50
112720/112720 [==============================] - 4s - loss: 0.6489 - acc: 0.6267 - val_loss: 1.6706 - val_acc: 0.6509
Epoch 9/50
112720/112720 [==============================] - 4s - loss: 0.5855 - acc: 0.7274 - val_loss: 1.66

112720/112720 [==============================] - 2s - loss: 0.2382 - acc: 0.8753 - val_loss: 1.0224 - val_acc: 0.8480
Epoch 25/50
112720/112720 [==============================] - 3s - loss: 0.2738 - acc: 0.8658 - val_loss: 1.1902 - val_acc: 0.8160
Epoch 26/50
112720/112720 [==============================] - 3s - loss: 0.3014 - acc: 0.8417 - val_loss: 1.1514 - val_acc: 0.7944
Epoch 27/50
112720/112720 [==============================] - 3s - loss: 0.2119 - acc: 0.8854 - val_loss: 1.1474 - val_acc: 0.7768
Epoch 28/50
112720/112720 [==============================] - 3s - loss: 0.2458 - acc: 0.8686 - val_loss: 1.1391 - val_acc: 0.7939
Epoch 29/50
112720/112720 [==============================] - 3s - loss: 0.2201 - acc: 0.8802 - val_loss: 1.0754 - val_acc: 0.8303
Epoch 30/50
112720/112720 [==============================] - 3s - loss: 0.1913 - acc: 0.8921 - val_loss: 1.0591 - val_acc: 0.8270
Epoch 31/50
112720/112720 [==============================] - 3s - loss: 0.1989 - acc: 0.8899 - val_los

In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
2,50,32,2,0.926544,0.926721
3,50,4,6,0.940649,0.894163
5,50,32,6,0.910309,0.893630
8,50,32,10,0.921664,0.890170
4,50,16,6,0.907115,0.887731
0,50,4,2,0.899574,0.873093
7,50,16,10,0.890703,0.831529
6,50,4,10,0.877129,0.797773
1,50,16,2,0.363290,0.335034


In [6]:
scores.to_pickle("dataset/vae_only_feature_extraction_scores.pkl")